![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F02+-+Vertex+AI+AutoML&file=02Tools+-+AutoML+Cloud+Logging.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/02%20-%20Vertex%20AI%20AutoML/02Tools%20-%20AutoML%20Cloud%20Logging.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F02%2520-%2520Vertex%2520AI%2520AutoML%2F02Tools%2520-%2520AutoML%2520Cloud%2520Logging.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/02%20-%20Vertex%20AI%20AutoML/02Tools%20-%20AutoML%20Cloud%20Logging.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/02%20-%20Vertex%20AI%20AutoML/02Tools%20-%20AutoML%20Cloud%20Logging.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 02 Tools - AutoML Cloud Logging

## WORK IN PROGRESS

Use the Vertex AI Python Client parse through the AutoML Tuning and Model Ensemble logs.

### View Model Architeture in Cloud Logging


This [link](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/logging#before_you_begin) provides information on how to use Cloud Logging to view details about a Vertex AI model.

**Note**: By Default, logs are deleted after ***30 days***.

**Prerequisites:**
-  02b - Vertex AI - AutoML with clients (code)

---

## Setup

inputs:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '02b'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

# BigQuery destination 
BQ_PROJECT = PROJECT_ID
BQ_DATASET = '02tools_automl_cloud_logging'
BQ_TABLE_TUNING_LOGS = 'tuning-logs'
BQ_TABLE_MODEL_LOGS = 'model-logs'

packages:

In [ ]:
from google.cloud import aiplatform
from datetime import datetime
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd
import google.cloud.logging_v2 as logging
import pandas_gbq as pd_gbq
import matplotlib.pylab as plt

clients:

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [ ]:
model_list_filter = f"labels.notebook=\"{NOTEBOOK}\""
model_list_filter

In [ ]:
model_list = aiplatform.Model.list(filter=str(model_list_filter))
create = [m.create_time for m in model_list] 
create.index(max(create))
model_run_date_time = model_list[0].create_time
model_create_time = model_list[0].create_time
model_run_ts = datetime.strptime(str(model_create_time),"%Y-%m-%d %H:%M:%S.%f%z")
model_run_ts = model_run_ts.strftime("%Y-%m-%dT%H:%M:%SZ")
model_run_ts

In [ ]:
# # import google.cloud.logging_v2 as logging
# # import pandas as pd
# # import json
# # import matplotlib.pylab as plt
# def get_logs(log_level):
#     logging_client = logging.Client()
#     hp_list = []
#     FILTER = log_filter
#     entries = logging_client.list_entries(filter_=FILTER)
#     for ind, entry in enumerate(entries):
#         if type(entry.payload) != dict:
#             continue
#         parse_log = entry.to_api_repr()
#         if model_log_filter in parse_log["logName"]:
#             if log_level == "tuning":
#                 for hp in parse_log["jsonPayload"]["modelStructure"]["modelParameters"]:
#                     # print(hp["hyperparameters"])
#                     hp_dict = hp["hyperparameters"]
#                     hp_dict["training_objective_point"] = parse_log['jsonPayload']['trainingObjectivePoint']['value']
#                     hp_list.append(hp_dict)
#             elif log_level == "model":
#                 for hp in parse_log["jsonPayload"]["modelParameters"]:
#                     hp_dict = hp["hyperparameters"]
#                     hp_list.append(hp_dict)
#             df = pd.DataFrame(hp_list)
#     return df

# display(get_logs(log_level))

---

### Parsing AutoML logs using Cloud Logging API

This [link](https://cloud.google.com/python/docs/reference/logging/latest/index.html) contains the details about using the **python client** for cloud logging.

There are two log levels in AutoML:
- Tuning
- Model

To start with, we will retrieve the tuning logs by setting the **log level** as `tuning` in the parameter list below:


parameters:

In [ ]:
log_level = "tuning"
model_log_filter = f"projects/{PROJECT_ID}/logs/automl.googleapis.com%2F{log_level}"
model_log_filter
log_filter = f"timestamp > \"{model_run_ts}\" resource.type=\"cloudml_job\""
model_log_filter
log_filter

Function to fetch tuning logs:

In [ ]:
def get_tuning_logs(log_level):
    logging_client = logging.Client()
    hp_list = []
    FILTER = log_filter
    entries = logging_client.list_entries(filter_=FILTER)
    for ind, entry in enumerate(entries):
        if type(entry.payload) != dict:
            continue
        parse_log = entry.to_api_repr()
        if model_log_filter in parse_log["logName"]:
            for hp in parse_log["jsonPayload"]["modelStructure"]["modelParameters"]:
                hp_dict = hp["hyperparameters"]
                hp_list.append(hp_dict)
            tuning_log_df = pd.DataFrame(hp_list)
    return tuning_log_df

In [ ]:
tuning_log_df = get_tuning_logs(log_level)
tuning_log_df = tuning_log_df.applymap(str)
# tuning_log_df.to_gbq(destination_table="automl_log.tuning_logs")
# pd_gbq.to_gbq(tuning_log_df,"automl_log.tuning_logs_str",project_id=PROJECT_ID)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
  display(tuning_log_df)

Next step is to add **training objective point** to the tuning logs. Please refer the [link](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/logging#before_you_begin) to learn more about training objective point.

In [ ]:
def get_tuning_logs_with_obj(log_level):
    logging_client = logging.Client()
    hp_list = []
    FILTER = log_filter
    entries = logging_client.list_entries(filter_=FILTER)
    for ind, entry in enumerate(entries):
        if type(entry.payload) != dict:
            continue
        parse_log = entry.to_api_repr()
        if model_log_filter in parse_log["logName"]:
            for hp in parse_log["jsonPayload"]["modelStructure"]["modelParameters"]:
                # print(hp["hyperparameters"])
                hp_dict = hp["hyperparameters"]
                hp_dict["training_objective_point"] = parse_log['jsonPayload']['trainingObjectivePoint']['value']
                hp_list.append(hp_dict)
            df = pd.DataFrame(hp_list)
    return df

tuning_with_obj_log_df = get_tuning_logs_with_obj(log_level)
# pd_gbq.to_gbq(tuning_with_obj_log_df,"automl_log.tuning_with_obj_logs",project_id=PROJECT_ID)

For retrieving the model ensemble logs, update the **log_level** to `model` in the parameter list below:

In [ ]:
log_level = "model"
model_log_filter = "projects/" + PROJECT_ID + "/logs/automl.googleapis.com%2F" + log_level
model_log_filter
log_filter = "timestamp > \"" + model_run_ts + "\" resource.type=\"cloudml_job\""
model_log_filter
log_filter

In [ ]:
def get_model_logs(log_level):
    logging_client = logging.Client()
    hp_list = []
    FILTER = log_filter
    entries = logging_client.list_entries(filter_=FILTER)
    for ind, entry in enumerate(entries):
        if type(entry.payload) != dict:
            continue
        parse_log = entry.to_api_repr()
        if model_log_filter in parse_log["logName"]:
            for hp in parse_log["jsonPayload"]["modelParameters"]:
                hp_dict = hp["hyperparameters"]
                hp_list.append(hp_dict)
            model_log_df = pd.DataFrame(hp_list)
    return model_log_df

# display(get_model_logs(log_level))

In [ ]:
model_log_df = get_model_logs(log_level)
model_log_df = model_log_df.applymap(str)
# pd_gbq.to_gbq(model_log_df,"automl_log.model_logs",project_id=PROJECT_ID)
# display(model_log_df)

Joining **Tuning** and **Model** logs to display the common training parmeters present in model ensemble logs

In [ ]:
df_cd = pd.merge(tuning_log_df, model_log_df, how='inner')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_cd)

---

## Loading logs into BigQuery for analysis

List BigQuery datasets in the project. Check for dataset name `02tools_automl_cloud_logging`

In [ ]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

: 

Create the dataset if missing:

In [ ]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds = bq.create_dataset(dataset = ds, exists_ok = True)

Validating the newly created BigQuery dataset:

In [ ]:
datasets = list(bigquery.list_datasets())
for d in datasets:
    print(d.dataset_id)

Loading Tuning Logs to BigQuery Table:

In [ ]:
pd_gbq.to_gbq(tuning_log_df,f"{BQ_DATASET}.{BQ_TABLE_TUNING_LOGS}",project_id=PROJECT_ID)

Loading Model Logs to BigQuery Table:

In [ ]:
pd_gbq.to_gbq(model_log_df,f"{BQ_DATASET}.{BQ_TABLE_MODEL_LOGS}",project_id=PROJECT_ID)

Retrieve the matching rows from tuning and model tables in BigQuery:

In [ ]:
query = f"""
SELECT * 
FROM 
`{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_TUNING_LOGS}`
INNER JOIN
`{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_MODEL_LOGS}`
using 
(enable_numerical_embeddings ,
enable_embedding_l2 ,
dropout,
enable_batch_norm,
enable_l1,
enable_embedding_l1,
num_hidden_layers,
normalized_numerical,
skip_connections_type,
num_cross_layers,
enable_l2,
model_type,
enable_layer_norm ,
hidden_layer_size ,
max_tree_depth ,
num_trees ,
l2 ,
tree_complexity ,
l1);
"""

: 

Displaying the common training parameters using BigQuery:

In [ ]:
data = bq.query(query = query).to_dataframe()
data

---
### Reusable function to retrieve AutoML tuning and model logs
Adding the function for reference purpose:

In [ ]:
def get_logs(log_level):
    logging_client = logging.Client()
    hp_list = []
    FILTER = log_filter
    entries = logging_client.list_entries(filter_=FILTER)
    for ind, entry in enumerate(entries):
        if type(entry.payload) != dict:
            continue
        parse_log = entry.to_api_repr()
        if model_log_filter in parse_log["logName"]:
            if log_level == "tuning":
                for hp in parse_log["jsonPayload"]["modelStructure"]["modelParameters"]:
                    # print(hp["hyperparameters"])
                    hp_dict = hp["hyperparameters"]
                    hp_dict["training_objective_point"] = parse_log['jsonPayload']['trainingObjectivePoint']['value']
                    hp_list.append(hp_dict)
            elif log_level == "model":
                for hp in parse_log["jsonPayload"]["modelParameters"]:
                    hp_dict = hp["hyperparameters"]
                    hp_list.append(hp_dict)
            df = pd.DataFrame(hp_list)
    return df